# 1. Imports

In [3]:
import json
import math
import scipy
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics 
import seaborn as sns
from numpy.random import randn
from statistics import mode
%matplotlib inline

# 2. Load Data
## 2.1 Export Data from Database as Json-Files

In [2]:
!./data/raw_data/thesisDataExport.sh

2021-07-19T14:41:28.699+0200	connected to: mongodb+srv://[**REDACTED**]@pearup.r81v8.mongodb.net/pearup
2021-07-19T14:41:29.311+0200	exported 435 records
2021-07-19T14:41:29.834+0200	connected to: mongodb+srv://[**REDACTED**]@pearup.r81v8.mongodb.net/pearup
2021-07-19T14:41:29.963+0200	exported 48 records
2021-07-19T14:41:30.489+0200	connected to: mongodb+srv://[**REDACTED**]@pearup.r81v8.mongodb.net/pearup
2021-07-19T14:41:31.514+0200	[........................]  pearup.pupils  0/1508  (0.0%)
2021-07-19T14:41:32.515+0200	[........................]  pearup.pupils  0/1508  (0.0%)
2021-07-19T14:41:33.513+0200	[........................]  pearup.pupils  0/1508  (0.0%)
2021-07-19T14:41:34.512+0200	[........................]  pearup.pupils  0/1508  (0.0%)
2021-07-19T14:41:35.514+0200	[........................]  pearup.pupils  0/1508  (0.0%)
2021-07-19T14:41:35.618+0200	[########################]  pearup.pupils  1508/1508  (100.0%)
2021-07-19T14:41:35.618+0200	exported 1508 records
2021-07-19T

## 2.2 Load Json-Files into python dictionaries

In [2]:
pupils = json.load(open('data/raw_data/pupils.json', 'r'))
classrooms = json.load(open('data/raw_data/classrooms.json', 'r'))
taskSeries = json.load(open('data/raw_data/taskSeries.json', 'r'))

print("Anzahl SuS: ", len(pupils))
print("Anzahl Klassen: ", len(classrooms))

Anzahl SuS:  1454
Anzahl Klassen:  428


# 3. Select the correct pupils and classes
## 3.1 Select all pupils that worked on the AI tasks

For each pupil, go over every task they did and check whether they did one of the AI tasks. If they did, add them and their class to the respective lists.

In [27]:
# taskseriesIDs vom KI Paket:
KI_taskseries_ids = ["AI-1-v2", "AI-2-v2", "AI-3-v2", "AI-4-v2", "ML-1", "ML-2", "ML-3", "Classification-1", "Clustering-1", "Classification-2", "Clustering-2", "LimitsML", "EvalML", "Impact-1", "Impact-2", "Impact-3", "Impact-4", "AIClosing"]

classIDsKI = []
pupilsKI = []

for pupilnumber in range(len(pupils)): #für jeden schüler
    for tasknumber in range(len(pupils[pupilnumber]["solvedTaskSeries"])): #für jede seiner tasks
        if("taskSeriesId" in pupils[pupilnumber]["solvedTaskSeries"][tasknumber].keys()):
            if(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["taskSeriesId"] in KI_taskseries_ids):
                for sectionnumber in range(len(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"])): #für diese tasks, alle sections
                    for subsectionnumber in range(len(pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"][sectionnumber]["currentSubSections"])):
                        if("exercise" in pupils[pupilnumber]["solvedTaskSeries"][tasknumber]["currentSections"][sectionnumber]["currentSubSections"][subsectionnumber].keys()):
                            if pupils[pupilnumber]["classId"] not in classIDsKI:
                                classIDsKI.append(pupils[pupilnumber]["classId"])
                            if pupils[pupilnumber] not in pupilsKI:
                                pupilsKI.append(pupils[pupilnumber])

## 3.2 Select all classes with enough students that have worked on the AI tasks
This gets rid of internal test classes with only one or two pupils.

In [28]:
KI_classes = {classID: {"count": 0} for classID in classIDsKI}

for pupil in pupilsKI:
    KI_classes[pupil["classId"]]["count"] += 1

print(KI_classes)
print("")

# Jetzt nur die Klassen mit genügend SuS auswählen
final_pupils_KI = []
min_class_size = 10

for pupil in pupilsKI:
    if(KI_classes[pupil["classId"]]["count"] >= min_class_size):
        final_pupils_KI.append(pupil)

print("Zur Zeit gibt es " + str(len(final_pupils_KI)) + " sus, die das KI Paket bearbeitet haben")


{'uxfuiLT4xZA6yvE6L': {'count': 2}, 'qFEyvypMn4sAwLwtc': {'count': 26}, 'y7j9eKDPsqNkAycuy': {'count': 1}, 'XdLWZx5FrxRsoKap8': {'count': 2}, '6Z2MmkKQKebb7nz8P': {'count': 3}, '4fyXxCKaLbCBaPbYE': {'count': 1}, 'xYWgeSPkADwSmhdmm': {'count': 4}, 'vS8gpLSA8Y46fzWSN': {'count': 7}, 'GkQSJvPTRWsLnpyR9': {'count': 14}, '7cWBWtLEcLSYDygDc': {'count': 2}, 'kCmX662oevP4hJpSZ': {'count': 1}, 'GbPhM46pBxWirBfin': {'count': 1}, 'dJR3L7tcMwMRSydWE': {'count': 1}, 'TeQTDWWjCiMxJgNfg': {'count': 7}, 'hZ9Hi9DLZSm5JMbNm': {'count': 11}, 'Zx6YpkyjLyHd8Rmvp': {'count': 4}, 'NCm8NNbAmNsur5S33': {'count': 22}, 'Xr2bFE3qSrJk5EsNH': {'count': 1}, 'xskRD6rct9ASrjR9u': {'count': 1}, 'L4KttRnMpBPMidCfQ': {'count': 1}, 'E6MAFLQAPpbaq56Lq': {'count': 2}, 'xXcRs55AwSdHJwNqx': {'count': 2}, '7XiJ56zCm4omf95iZ': {'count': 2}, 'Xr7LXctGQp5cWjsRx': {'count': 3}, 'Zpjvpb2oZgE55M78Q': {'count': 1}, 'xgXvFFY7n6iizbrhM': {'count': 3}, '2xefmoTfmYkhhAZYp': {'count': 15}, 'eLnTroCFjgx2oTztm': {'count': 1}, 'wDSAN37T9Fn68

# 4. Transform the Data
## 4.1 Create a DF containing which pupil is in which class (class_df)

In [29]:
class_list = [{"pupilId": pupil["_id"], "classId": pupil["classId"]} for pupil in final_pupils_KI]
class_df = pd.DataFrame(class_list)

class_df.head()

,pupilId,classId
0,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc
1,Z9DkQMmtiPiHtteME,qFEyvypMn4sAwLwtc
2,sqBXKssFvS8pFYAMv,qFEyvypMn4sAwLwtc
3,Kd6FuRsrq6QvPrkKp,qFEyvypMn4sAwLwtc
4,jRT6yzZ2ip5ab5ZFM,qFEyvypMn4sAwLwtc


## 4.2 Create a DF containing each submitted task (pupil_df)

Each row will be one submitted task by one pupil with all the corresponding data fields.

In [30]:
tasktypes = ["blockly", "multipleChoice", "freetext", "survey", "inputField", "surveySlider", "noTask"]
exercise_keys = ['state', 'submitted', 'subType', 'evaluation', 'falseAttempts', 'hintsTaken', 'rating', 'teacherFeedback', "history"]

columns = ["pupilId", "taskId", "taskSeriesId", "duration"] + exercise_keys
pupil_df = pd.DataFrame(columns=columns)

row_nr = 0
for pupil in final_pupils_KI:
    for task in pupil["solvedTaskSeries"]:
        sec_nr = 1
        for section in task["currentSections"]:
            subsec_nr = 1
            for subsection in section["currentSubSections"]:
                # eine neue Zeile hinzufügen für jede bearbeitete Aufgabe und id columns füllen
                pupil_df = pupil_df.append({"pupilId": pupil["_id"], "taskId": str(task["taskSeriesId"])+"-"+str(sec_nr)+"-"+str(subsec_nr),'taskSeriesId': task["taskSeriesId"], "duration": subsection["duration"]}, ignore_index=True)
                subsec_nr += 1
                if("exercise" in subsection.keys()):
                    # hier wird der eigentliche state der aufgabe in die reihe eingetragen
                    for key in subsection["exercise"].keys():
                        pupil_df.at[row_nr, key] = subsection["exercise"][key]
                row_nr += 1
            sec_nr +=1
    
# aufgaben droppen, die noch nicht submitted wurden!
pupil_df.dropna(subset=['submitted'], inplace=True)

# die eine aufgabe mit der falschen lösung droppen
index_names = pupil_df[ pupil_df['taskId'] == "Classification-2-1-5" ].index 
pupil_df.drop(index_names, inplace = True) # drop the respective rows

pupil_df.head()


,pupilId,taskId,taskSeriesId,duration,state,submitted,subType,evaluation,falseAttempts,hintsTaken,rating,teacherFeedback,history,hints,revision,helpRequested
0,Ay6mfswm3SFcQdNJg,AI-1-v2-1-1,AI-1-v2,95,62,True,surveySlider,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
1,Ay6mfswm3SFcQdNJg,AI-1-v2-1-2,AI-1-v2,401,<p>Künstliche Intelligenz ist meiner Meinung n...,True,freetext,correct,NaN,NaN,"{'content': 5, 'format': 5, 'creativity': 5}",,[],NaN,NaN,NaN
2,Ay6mfswm3SFcQdNJg,AI-1-v2-2-1,AI-1-v2,337,"[0, 0, 0, 1, 0]",True,survey,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
4,Ay6mfswm3SFcQdNJg,AI-2-v2-1-1,AI-2-v2,242,"[True, True, False, True]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN
6,Ay6mfswm3SFcQdNJg,AI-3-v2-1-1,AI-3-v2,68,"[False, False, False]",True,multipleChoice,correct,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN


## 4.3 Create a DF containing all possible tasks (task_df)

Each row will be corresponding to one of the created tasks.

In [44]:
interaction_subtypes = ['multipleChoice', 'blockly', 'surveySlider', 'survey', 'inputField', 'freetext']
columns = ["taskId", "subType", "question", "answers", "minLabel", "maxLabel", "hints", "sampleSolution"]
task_df = pd.DataFrame(columns=columns)

row_nr = 0
for taskSerie in taskSeries:
    sec_nr = 1
    if taskSerie["_id"] in KI_taskseries_ids:
        for section in taskSerie["sections"]:
            subsec_nr = 1
            for subsection in section["subSections"]:
                for elem in subsection["elements"]:
                    if(elem["subType"] in interaction_subtypes):
                        task_df = task_df.append({"taskId": str(taskSerie["_id"])+"-"+str(sec_nr)+"-"+str(subsec_nr), "subType": elem["subType"], "question": elem["question"]["de"]}, ignore_index=True)
                        if(elem["subType"] == "surveySlider"):
                            task_df.at[row_nr, "minLabel"] = elem["specificData"]["minLabel"]["de"]
                            task_df.at[row_nr, "maxLabel"] = elem["specificData"]["maxLabel"]["de"]
                        elif(elem["subType"] == "freetext"):
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["de"]
                        elif(elem["subType"] == "survey"):
                            task_df.at[row_nr, "answers"] = elem["specificData"]["answers"]
                        elif(elem["subType"] == "multipleChoice"):
                            task_df.at[row_nr, "answers"] = elem["specificData"]["answers"]
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["universal"]
                            if(len(elem["hints"]) > 0):
                                task_df.at[row_nr, "hints"] = elem["hints"]#[0]["de"]
                        elif(elem["subType"] == "inputField"):
                            task_df.at[row_nr, "sampleSolution"] = elem["sampleSolution"]["universal"]
                            if(len(elem["hints"]) > 0):
                                task_df.at[row_nr, "hints"] = elem["hints"]#[0]["de"]
                        row_nr += 1
                subsec_nr +=1
            sec_nr +=1
            
task_df.set_index('taskId', inplace=True)
task_df.head()

,subType,question,answers,minLabel,maxLabel,hints,sampleSolution
taskId,,,,,,,
Clustering-2-1-1,freetext,Schau dir die Mittelpunkte der Cluster genau a...,NaN,NaN,NaN,NaN,Make-Up: Cluster 1 repräsentiert es am besten....
AI-1-v2-1-1,surveySlider,"Wie gut weißt du schon, was KI ist?",NaN,Gar nicht,Sehr gut,NaN,NaN
AI-1-v2-1-2,freetext,Was ist deiner Meinung nach Künstliche Intelli...,NaN,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...
AI-1-v2-2-1,survey,Wie viele deiner Zeichnungen hat die KI erkann...,"[{'de': '0', 'en-US': '0'}, {'de': '1-2', 'en-...",NaN,NaN,NaN,NaN
AI-2-v2-1-1,multipleChoice,"Kannst du erkennen, wo KI überall drin steckt?...",[{'de': 'Der Newsfeed eines Social Network nut...,NaN,NaN,"[{'de': 'Denk an die Beispiele, die wir uns am...","[True, True, False, True]"


## 4.4 Merge the corresponding DFs

In [45]:
new_df = pd.merge(pupil_df.drop(['subType', "hints"], axis = 1), task_df, on='taskId')
new_df.drop(columns=['history', "submitted"], inplace=True)
df = pd.merge(class_df, new_df, on="pupilId")
df.head()

,pupilId,classId,taskId,taskSeriesId,duration,state,evaluation,falseAttempts,hintsTaken,rating,teacherFeedback,revision,helpRequested,subType,question,answers,minLabel,maxLabel,hints,sampleSolution
0,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-1-v2-1-1,AI-1-v2,95,62,NaN,NaN,NaN,NaN,NaN,NaN,NaN,surveySlider,"Wie gut weißt du schon, was KI ist?",NaN,Gar nicht,Sehr gut,NaN,NaN
1,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-1-v2-1-2,AI-1-v2,401,<p>Künstliche Intelligenz ist meiner Meinung n...,correct,NaN,NaN,"{'content': 5, 'format': 5, 'creativity': 5}",,NaN,NaN,freetext,Was ist deiner Meinung nach Künstliche Intelli...,NaN,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...
2,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-1-v2-2-1,AI-1-v2,337,"[0, 0, 0, 1, 0]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,survey,Wie viele deiner Zeichnungen hat die KI erkann...,"[{'de': '0', 'en-US': '0'}, {'de': '1-2', 'en-...",NaN,NaN,NaN,NaN
3,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-2-v2-1-1,AI-2-v2,242,"[True, True, False, True]",correct,NaN,NaN,NaN,NaN,NaN,NaN,multipleChoice,"Kannst du erkennen, wo KI überall drin steckt?...",[{'de': 'Der Newsfeed eines Social Network nut...,NaN,NaN,"[{'de': 'Denk an die Beispiele, die wir uns am...","[True, True, False, True]"
4,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-3-v2-1-1,AI-3-v2,68,"[False, False, False]",correct,NaN,NaN,NaN,NaN,NaN,NaN,multipleChoice,Was für Sachen können KI-Anwendungen?,"[{'de': 'Die Anwendung, die den Facebook-Feed ...",NaN,NaN,[{'de': 'Kann überhaupt eine dieser Anwendunge...,"[False, False, False]"


## 4.5 Preprocess entries of DFs
### 4.5.1 get rid of the p-tag in the freetext answers

In [46]:
mask_freetext = df['subType'].str.startswith('freetext')
df.loc[mask_freetext, 'state'] = df.loc[mask_freetext, 'state'].str.replace("<p>","")
df.loc[mask_freetext, 'state'] = df.loc[mask_freetext, 'state'].str.replace("</p>","")

### 4.5.2 add columns "number words in answer", "numberFalseAttempts" and "numberHints"

In [47]:
answer_length_array = []
numberFalseAttempts = []
numberHints=[]

for i, row in df.iterrows():
    if row["subType"] == "freetext":
        answer_length_array.append(int(len(row["state"].split())))
    else:
        answer_length_array.append(np.nan)
    if row["falseAttempts"] != row["falseAttempts"]:
        numberFalseAttempts.append(0)
    else:
        numberFalseAttempts.append(len(row["falseAttempts"]))
    if row["hints"] != row["hints"]:
        numberHints.append(np.nan)
    else:
        numberHints.append(len(row["hints"]))

df["number words in answer"] = answer_length_array
df["numberFalseAttempts"] = numberFalseAttempts
df["numberHints"] = numberHints

### 4.5.3 add columns for the teacher ratings of the freetext answers

In [48]:
rating_content = []
rating_format = []
rating_creativity = []

for i, task in df.iterrows():
    if((task["subType"] == "freetext") & (task["evaluation"] != "pending")):
        rating_content.append(task["rating"]["content"])
        rating_format.append(task["rating"]["format"])
        rating_creativity.append(task["rating"]["creativity"])
    else:
        rating_content.append(np.nan)
        rating_format.append(np.nan)
        rating_creativity.append(np.nan)
        
        

df["rating_content"] = rating_content    
df["rating_format"] = rating_format 
df["rating_creativity"] = rating_creativity

df.head()

,pupilId,classId,taskId,taskSeriesId,duration,state,evaluation,falseAttempts,hintsTaken,rating,...,minLabel,maxLabel,hints,sampleSolution,number words in answer,numberFalseAttempts,numberHints,rating_content,rating_format,rating_creativity
0,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-1-v2-1-1,AI-1-v2,95,62,NaN,NaN,NaN,NaN,...,Gar nicht,Sehr gut,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
1,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-1-v2-1-2,AI-1-v2,401,Künstliche Intelligenz ist meiner Meinung nach...,correct,NaN,NaN,"{'content': 5, 'format': 5, 'creativity': 5}",...,NaN,NaN,NaN,Die Schülerinnen und Schüler (SuS) sollen besc...,119.0,0,NaN,5.0,5.0,5.0
2,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-1-v2-2-1,AI-1-v2,337,"[0, 0, 0, 1, 0]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-2-v2-1-1,AI-2-v2,242,"[True, True, False, True]",correct,NaN,NaN,NaN,...,NaN,NaN,"[{'de': 'Denk an die Beispiele, die wir uns am...","[True, True, False, True]",NaN,0,1.0,NaN,NaN,NaN
4,Ay6mfswm3SFcQdNJg,qFEyvypMn4sAwLwtc,AI-3-v2-1-1,AI-3-v2,68,"[False, False, False]",correct,NaN,NaN,NaN,...,NaN,NaN,[{'de': 'Kann überhaupt eine dieser Anwendunge...,"[False, False, False]",NaN,0,1.0,NaN,NaN,NaN


## 4.6 Create a DF containing performance measures for each pupil
### 4.6.1 create fields for number of completed tasks, total duration and average duration

In [57]:
nr_tasks_per_pupil = []
total_duration_per_pupil = []

for pupilId in df["pupilId"].unique():
    nr_tasks_per_pupil.append(len(df[df["pupilId"]==pupilId]))
    total_duration_per_pupil.append(sum(df[df["pupilId"]==pupilId]["duration"])/60)

pupil_performance = {"pupilId": df["pupilId"].unique(), "nr_tasks_per_pupil": nr_tasks_per_pupil, "total_duration in minutes": total_duration_per_pupil}
pupil_performance_df = pd.DataFrame(pupil_performance)
pupil_performance_df["average duration per task in minutes"] = pupil_performance_df["total_duration in minutes"]/pupil_performance_df["nr_tasks_per_pupil"]

pupil_performance_df = pupil_performance_df.set_index('pupilId')

pupil_performance_df.head()

,nr_tasks_per_pupil,total_duration in minutes,average duration per task in minutes
pupilId,,,
Ay6mfswm3SFcQdNJg,66,120.116667,1.819949
Z9DkQMmtiPiHtteME,4,25.950000,6.487500
sqBXKssFvS8pFYAMv,66,66.650000,1.009848
Kd6FuRsrq6QvPrkKp,6,37.383333,6.230556
jRT6yzZ2ip5ab5ZFM,66,66.566667,1.008586


### 4.6.2 For each freetext task add fields for teacher ratings and own ratings of the answers

In [58]:
taskIds = ["AI-1-v2-1-1", "AI-4-v2-1-4", "EvalML-1-1", "ML-2-1-1", 'EvalML-1-5', "AI-4-v2-1-5", 'EvalML-1-2', 'EvalML-1-6', 'AIClosing-1-1', 'AIClosing-1-3', 'AIClosing-1-4', "AI-1-v2-1-2", 'AI-4-v2-1-3', 'ML-2-1-2', 'EvalML-1-7', 'AIClosing-1-2', 'AIClosing-1-5', 'LimitsML-1-2', "AI-2-v2-1-1", 'EvalML-1-4', 'AI-4-v2-1-1', 'AI-4-v2-1-2', 'AI-3-v2-1-2', 'EvalML-1-3', 'Classification-1-1-1', "Clustering-1-1-1", 'ML-3-1-2', 'ML-3-1-3','ML-3-1-4','EvalML-1-9', 'Classification-2-1-8', "Impact-1-1-1", 'EvalML-1-8', 'LimitsML-1-1', 'AIClosing-1-6','AIClosing-1-7','AIClosing-1-8']

for taskId in taskIds:
    pupil_performance_df[taskId] = np.nan
    if(task_df.loc[taskId, "subType"] == "freetext"):
        for pointType in ["_eval", "_null_answer", "_unknown", "_correct_points", "_incorrect_points", "_irrelevant_points"]:
            pupil_performance_df[taskId+pointType] = np.nan
        for ratingType in ["rating_content", "rating_format", "rating_creativity"]:
            pupil_performance_df[taskId+"_"+ratingType] = np.nan

### 4.6.3 Add additional fields the submitted answers of all tasktypes

In [59]:
for pupilId in pupil_performance_df.index:
    for taskId in taskIds:
        if(taskId in df[df["pupilId"] == pupilId]["taskId"].unique()):
            if (df[(df["pupilId"] == pupilId) & (df["taskId"] == taskId)][["subType"]].values == "surveySlider"):
                pupil_performance_df.loc[pupilId, taskId] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0]
            elif((df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["subType"]].values == "freetext") & (df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["evaluation"]].values == "correct")):
                pupil_performance_df.loc[pupilId, taskId] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_eval"] = "correct"
                pupil_performance_df.loc[pupilId, taskId+"_rating_content"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_content"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_format"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_format"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_creativity"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_creativity"]].values[0][0]
            elif((df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["subType"]].values == "freetext") & (df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["evaluation"]].values == "pending")):
                pupil_performance_df.loc[pupilId, taskId] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_eval"] = "pending"
                pupil_performance_df.loc[pupilId, taskId+"_rating_content"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_content"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_format"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_format"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_creativity"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_creativity"]].values[0][0]
            elif((df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["subType"]].values == "freetext") & (df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["evaluation"]].values == "false")):
                pupil_performance_df.loc[pupilId, taskId] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_eval"] = "false"
                pupil_performance_df.loc[pupilId, taskId+"_rating_content"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_content"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_format"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_format"]].values[0][0]
                pupil_performance_df.loc[pupilId, taskId+"_rating_creativity"] = df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["rating_creativity"]].values[0][0]
            elif(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["subType"]].values == "multipleChoice"):
                if(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values == df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values):
                    pupil_performance_df.loc[pupilId, taskId] = str(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values[0][0][0])
                elif(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values != df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["falseAttempts"]].values):
                    pupil_performance_df.loc[pupilId, taskId] = str(df[(df["pupilId"] == pupilId)  & (df["taskId"] == taskId)][["state"]].values[0][0])
        elif((taskId not in df[df["pupilId"] == pupilId]["taskId"].unique()) & (task_df.loc[taskId, "subType"] == "multipleChoice")):
            pupil_performance_df.loc[pupilId, taskId] = str([])
    
mcs = ["AI-2-v2-1-1", 'EvalML-1-4', 'AI-4-v2-1-1', 'AI-4-v2-1-2', 'AI-3-v2-1-2', 'EvalML-1-3', 'Classification-1-1-1', "Clustering-1-1-1", 'ML-3-1-2', "ML-3-1-3", "ML-3-1-4", 'EvalML-1-9', 'Classification-2-1-8', "Impact-1-1-1", 'EvalML-1-8', 'LimitsML-1-1']
for mcTaskId in mcs:
    pupil_performance_df[mcTaskId] = pupil_performance_df[mcTaskId].apply(ast.literal_eval)

for pupilId in pupil_performance_df.index:
    pupil_performance_df.loc[pupilId, "classId"] = df[df["pupilId"] == pupilId]["classId"].unique()[0]

pupil_performance_df.drop(["5GPMWTgF29JDyeXoG"], inplace=True) #5GPMWTgF29JDyeXoG droppen, vermutlich nur lehrer, der hochspielt    

pupil_performance_df.head()


,nr_tasks_per_pupil,total_duration in minutes,average duration per task in minutes,AI-1-v2-1-1,AI-4-v2-1-4,EvalML-1-1,ML-2-1-1,EvalML-1-5,AI-4-v2-1-5,EvalML-1-2,...,AIClosing-1-8_eval,AIClosing-1-8_null_answer,AIClosing-1-8_unknown,AIClosing-1-8_correct_points,AIClosing-1-8_incorrect_points,AIClosing-1-8_irrelevant_points,AIClosing-1-8_rating_content,AIClosing-1-8_rating_format,AIClosing-1-8_rating_creativity,classId
pupilId,,,,,,,,,,,,,,,,,,,,,
Ay6mfswm3SFcQdNJg,66,120.116667,1.819949,62.0,91.0,81.0,71.0,75.0,90.0,90.0,...,pending,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
Z9DkQMmtiPiHtteME,4,25.950000,6.487500,43.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
sqBXKssFvS8pFYAMv,66,66.650000,1.009848,89.0,99.0,96.0,79.0,93.0,100.0,98.0,...,correct,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,qFEyvypMn4sAwLwtc
Kd6FuRsrq6QvPrkKp,6,37.383333,6.230556,78.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,qFEyvypMn4sAwLwtc
jRT6yzZ2ip5ab5ZFM,66,66.566667,1.008586,90.0,88.0,76.0,71.0,69.0,72.0,100.0,...,correct,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,qFEyvypMn4sAwLwtc


## 4.7 Split DF based on tasktypes

In [60]:
surveySlider = df[df["subType"]=="surveySlider"].copy()
surveySlider.drop(["hints", "hintsTaken", "numberFalseAttempts", "sampleSolution", "answers", "falseAttempts", "rating", "teacherFeedback", "number words in answer", "rating_content", "rating_format", "rating_creativity"], axis = 1, inplace=True) 


In [61]:
inputFields = df[df["subType"]=="inputField"].copy()
inputFields.drop(["minLabel", "maxLabel", "answers", "rating", "teacherFeedback", "number words in answer", "rating_content", "rating_format", "rating_creativity"], axis = 1, inplace=True) 


In [62]:
multipleChoice = df[df["subType"]=="multipleChoice"].copy()
multipleChoice.drop(["minLabel", "maxLabel","rating", "teacherFeedback", "number words in answer", "rating_content", "rating_format", "rating_creativity"], axis = 1, inplace=True)


In [63]:
freetext = df[df["subType"] == "freetext"].copy()
freetext.drop(["minLabel", "maxLabel", "answers", "hints", "hintsTaken", "numberFalseAttempts", "rating"], axis = 1, inplace=True) 


# 5. Save the DFs

In [64]:
pupil_performance_df.to_pickle('data/processed/pupil_performance.pkl')
pupil_performance_df.to_csv('data/processed/pupil_performance.csv')
df.to_pickle('data/processed/df.pkl')
surveySlider.to_pickle('data/processed/surveySlider.pkl')
inputFields.to_pickle('data/processed/inputFields.pkl')
multipleChoice.to_pickle('data/processed/multipleChoice.pkl')
freetext.to_pickle('data/processed/freetext.pkl')
freetext.to_csv('data/processed/freetext.csv')
task_df.to_pickle('data/processed/task_df.pkl')